In [38]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np 
import cv2
import joblib
import matplotlib.pyplot as plt
%matplotlib inline

from script.helpers import convert, show_images, put_boxes, box_boundaries
from script.feature_source import FeatureExtracter
from script.model_classification import SVMObjectClassifier
from script.helpers import show_images
from script.slider import Slider
from script.heatmap import HeatMap 

In [39]:
import moviepy
moviepy.__version__

'1.0.3'

In [97]:
def hot_to_rgb(data):
    data_ = (data - np.min(data)) / (np.max(data) - np.min(data))
    my_cmap = plt.get_cmap('hot')
    img = my_cmap(data_)
    rgb_img = np.dstack((img[:, :, 0], img[:, :, 1], img[:, :, 2]))
    return rgb_img

In [98]:
model = SVMObjectClassifier()
model.load('./save_model')

sourcer_params = {
  'color_model': 'yuv',                # hls, hsv, yuv, ycrcb,
  'spatial_size': (64, 64),            # (16, 16), (32, 32), (64, 64)
  'orientations': 9,        # 6 - 12
  'pixels_per_cell': 8,               # 8, 16
  'cells_per_block': 2,                # 1, 2
  'transform_sqrt': True,
  'block_norm': 'L2',
  'hog_visualize': False
}
feature_extracter = FeatureExtracter(**sourcer_params)
model.set_feature_extracter(feature_extracter)
slider = Slider(model, None, (64, 64), 20, scale=1, strip_position=None, visualize=False)

In [89]:
def imresize(image, fraction):
    return cv2.resize(image, (int(image.shape[1] * fraction), int(image.shape[0] * fraction)))

In [114]:
temp_frame = cv2.imread('./test_images/test1.jpg')
heatmap = HeatMap(frame = temp_frame, thresh = 5, memory = 30)
def verbose_pipeline(this_frame, view_process = False):
  windowSizes = [(80, 80), (100, 100), (120, 120)]
  strip_positions = [(410, 600), (400, 600), (400, 600)]
  for window_size, strip_position in zip(windowSizes, strip_positions):
    slider.update_strip_position(strip_position)
    slider.update_window_size(window_size)
    bounding_boxes = slider.predict(this_frame, 0.7)
    heatmap.update(bounding_boxes)
    if view_process:
      print(bounding_boxes)
  
  mp, _, _ = heatmap.get()
  labeled_img = heatmap.draw(this_frame)
  rgb_img = imresize(hot_to_rgb(mp), 0.25)
  labeled_img[20:20 + rgb_img.shape[0], 20:20 + rgb_img.shape[1]] = rgb_img * 200
  if view_process
    show_images([rgb_img, labeled_img], 1, 2, W=15, H=7)
  return labeled_img

In [115]:
project_output = 'test_video_output.mp4'
clip1 = VideoFileClip("./video/test_video.mp4")
white_clip = clip1.fl_image(verbose_pipeline) 
%time white_clip.write_videofile(project_output, audio = False)

HTML("""
<video width="1060" height="640" controls>
  <source src="{0}">
</video>
""".format(project_output))

[[820, 410, 900, 490, 'vehicle', 0.7426827433321591], [860, 410, 940, 490, 'vehicle', 0.8128420320721418], [1000, 410, 1080, 490, 'vehicle', 0.7529445000750925]]
[[820, 400, 920, 500, 'vehicle', 0.877443617575847], [840, 400, 940, 500, 'vehicle', 0.8754287612679469]]


[[820, 400, 940, 520, 'vehicle', 0.8453589352713159]]
Moviepy - Building video test_video_output.mp4.
Moviepy - Writing video test_video_output.mp4



t:   0%|          | 0/38 [00:00<?, ?it/s, now=None]

[[820, 410, 900, 490, 'vehicle', 0.7426827433321591], [860, 410, 940, 490, 'vehicle', 0.8128420320721418], [1000, 410, 1080, 490, 'vehicle', 0.7529445000750925]]
[[820, 400, 920, 500, 'vehicle', 0.877443617575847], [840, 400, 940, 500, 'vehicle', 0.8754287612679469]]


t:   5%|▌         | 2/38 [00:57<17:17, 28.82s/it, now=None]

[[820, 400, 940, 520, 'vehicle', 0.8453589352713159]]
[[860, 410, 940, 490, 'vehicle', 0.8154966374641528], [1000, 410, 1080, 490, 'vehicle', 0.8184436931740763], [1020, 410, 1100, 490, 'vehicle', 0.7698907198080258]]


KeyboardInterrupt: 